In [1]:
# import required libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tensorflow.python.framework import ops
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib
ops.reset_default_graph()
sess = tf.InteractiveSession()

# Se cargan los datos

In [2]:
#sess = tf.Session()# Training loop
Xdf_raw = pd.read_csv("../IndoorFingerprint.csv").drop("Y", axis=1)
Ydf_raw = pd.read_csv("../IndoorFingerprint.csv").drop("X", axis=1)
x = tf.placeholder(tf.float32, shape=[None, 8])
y_ = tf.placeholder(tf.float32, shape=[None, 11])

# Se escalan los datos

In [3]:
# Normalizacion de los datos
atribute_numbers = Xdf_raw.shape[1]-1

# Valores para el dataset que predice X
Xtrx, Xtex, ytrx, ytex = train_test_split(
Xdf_raw.iloc[:, 1:], Xdf_raw["X"], test_size=0.2, random_state=0)
train_size = len(Xtrx)
test_size = len(Xtex)

# Valores para el dataset que predice Y
Xtry, Xtey, ytry, ytey = train_test_split(
Ydf_raw.iloc[:, 1:], Ydf_raw["Y"], test_size=0.2, random_state=0)

# Se normalizan los datos para ambos

scaler = preprocessing.StandardScaler()
Xtrx = scaler.fit_transform(Xtrx)
Xtex = scaler.transform(Xtex)

scaler = preprocessing.StandardScaler()
Xtry = scaler.fit_transform(Xtry)
Xtey = scaler.transform(Xtey)

# One Hot Vector

In [4]:
class_number_x = ytrx.unique().size
class_number_y = ytry.unique().size

ytrx_dummy = pd.get_dummies(ytrx)
ytex_dummy = pd.get_dummies(ytex)

ytry_dummy = pd.get_dummies(ytry)
ytey_dummy = pd.get_dummies(ytey)

# Se convierten valores a Tensores X

Xtrx = np.array(Xtrx,dtype='float32')
Xtex = np.array(Xtex,dtype='float32')
ytrx_dummy = np.array(ytrx_dummy,dtype='int32')
ytex_dummy = np.array(ytex_dummy,dtype='int32')

# Se convierten valores a Tensores Y

Xtry = np.array(Xtry,dtype='float32')
Xtey = np.array(Xtey,dtype='float32')
ytry_dummy = np.array(ytry_dummy,dtype='int32')
ytey_dummy = np.array(ytey_dummy,dtype='int32')

ytrx_cat = pd.Series(ytrx, dtype="category")
ytrx = np.array(ytrx_cat.cat.codes,dtype='int32')
ytex_cat = pd.Series(ytex, dtype="category")
ytex = np.array(ytex_cat.cat.codes,dtype='int32')

print(Xtrx.shape)
print(ytrx_dummy.shape)

(5280, 8)
(5280, 11)


In [5]:
x = tf.placeholder(tf.float32, shape=[None, atribute_numbers])
y_ = tf.placeholder(tf.float32, shape=[None, class_number_x])

In [6]:
W = tf.Variable(tf.zeros([atribute_numbers,class_number_x]))
b = tf.Variable(tf.zeros([class_number_x]))

In [7]:
sess.run(tf.global_variables_initializer())

In [8]:
y = tf.matmul(x,W) + b

In [9]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

In [10]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
batch_size = 100

In [11]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [12]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert inputs.shape[0] == targets.shape[0]
    if shuffle:
        indices = np.arange(inputs.shape[0])
        np.random.shuffle(indices)
    for start_idx in range(0, inputs.shape[0] - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [13]:
for _ in range(1000):
    #for batch in iterate_minibatches(Xtrx, ytrx_dummy, batch_size, shuffle=True):
    batch = next_batch(batch_size, Xtrx, ytrx_dummy)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [14]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: Xtex, y_: ytex_dummy}))

0.699242
